# Предвидување на варијатни на глико-протеинот на SARS-Cov2 вирусот
*** 
- **Ментор**: проф. д-р. Невена Ацковска
- **Студенти**: Кирил Зеленковски [161141]
- **Линк до Colab тетратка**: [Transformer-training.ipynb](https://colab.research.google.com/drive/1EyPGD7PAIAyEDOYtpmiKo07YJfQV5qKW?usp=sharing) 
<br> <br>


<p align="center">
<img src="https://raw.githubusercontent.com/zelenelez/images/master/finki.jpg" width=50%;></img> <br>
Летен семестар, 2021
</p>

## 1: Формирање на податочно множество

Како експеримент одлучив да додам уште еден месец од податоци и да видам како ќе се однесуваат бројките (повеќе како статистичка анализа) дали ќе се зголеми бројот на варијанти и да видам како моделот ќе учи со модифицираното множество. 



За таа цел, продолжуваме од Фаза 2 и Фаза 3 каде дефиниравме: 
- Извор на база податоци (GISAID) 
- Mета-податоци за секој месец (го додадов и последниот месец со цел во крајниот извештај да направам споредба): **Јули 2020 – Мај 2021**
- Значење на секоја од колоните (features) во овие податоци
- Вкупен број на варијанти: 
 - 1 анализа (до Април 2021): **3 365 варијанти**
 - 2 анализа (до Мај 2021): **10 016 варијанти**
- Методи и спремни модели кои само треба да ги раниме со секвенци (реченици)

Единствена разлика, е што ги додаваме сите **"Co-occuring"** промени за самите мутации кое како постапка го објаснувам подолу во експериментот. 


Сега продолжуваме со анализата, т.е. формирањето на податочно множество за нашиот втор трансформер кој потребно е да изгради речник слично што го градевме во Фаза 3. 

Ова податочно множество како и тоа, го градиме да има информации за влез (**текст фајл кој внатре во секој нов ред има ВАЛИДНА УНИКАТНА варијанта**) од кој потоа тој учи и потоа доколку му маскираме (скриеме) позиција ни дава нови можни позиции како резултат од своето учење. 

In [ ]:
!pip install biopython

     |████████████████████████████████| 2.3MB 8.1MB/s 


### 1.1. Вчитување на варијанти (стари + Месец Мај 2021)
Започнуваме со вчитување на сите нови фајлови:

In [ ]:
import os
import glob
import pandas as pd

# Читај ги сите фајлови од data-new фолдерот
os.chdir("/content/drive/MyDrive/data-new")

# Проверка за фајл и бришење на ист
filePath = '/content/drive/MyDrive/data-new/combined_data.csv'

if os.path.exists(filePath):
    os.remove(filePath)
else:
    print("Can not delete the file as it doesn't exists")

file_extension = '.csv'
all_filenames = [i for i in glob.glob(f'*{file_extension}')]

# Додавај време (timestamp) пред нивна конкатенација
all_files_modified = []
for file in all_filenames:
    df = pd.read_csv(file)
    timestamp = file.split('-')[1].split(".")[0]
    # print(timestamp)
    df.insert(8, "Timestamp", timestamp)
    all_files_modified.append(df)


# pd.concat: команда која по default спојува dataframes вертикално
combined_csv_data = pd.concat(all_files_modified)
combined_csv_data.head()

,Variant,Literature Ref,#Genomes,#Top Location,#Top Clade,#Top Lineage,Co-occurring Changes,#Co-occurring Changes,Timestamp,Δ#Loc(S),#aachanges(C),(SxC)
0,452R_478K_681R,NaN,31862,18839 England,31814 G,31454 B.1.617.2,"Spike_T19R, Spike_E156G, Spike_D950N",16,May_2021,141,3,423
1,69del_70del_346S_452R,NaN,608,79 England,603 GR,601 C.36.3,"Spike_S12F, Spike_Q677H, Spike_A899S",23,May_2021,50,4,200
2,5F_69del_70del_144del_501Y,NaN,16074,2123 England,15949 GRY,16056 B.1.1.7,"Spike_A570D, Spike_Y145del, Spike_P681H",20,May_2021,34,5,170
3,69del_70del_144del_484K_501Y,NaN,1502,216 Tyrol,1485 GRY,1489 B.1.1.7,"Spike_A570D, Spike_Y145del, Spike_P681H",21,May_2021,29,5,145
4,5F_69del_70del_144del_490L_501Y,NaN,1616,392 Nordjylland,1586 GRY,1610 B.1.1.7,"Spike_A570D, Spike_F490S, Spike_Y145del",23,May_2021,21,6,126


Сега разликата од овие нови (повеќе updated) податоци е што имаме нова колона. Гледаме дека новата колона се вика **"Co-occurring Changes"**. Значи сега имаме плус мутации на самиот геном, плус на оние од колоната **'Variant'** кои се значајни места кај се врзуваат антитела (значајни според одредени референци тоа е колоната **'Literature Ref'**), додека **"Co-occurring Changes"** се однесува на промени што настануваат во останатиот дел од кодните региони и плус дополнителни мутации во самиот геном (пр во *NSP12* или *N*).

Но, има голема грешка на дечките од GISAID. Проблемот е што овие податоци повторно не се целосни (поради што и пратив мејл кој го прикачив со фајловите). 

Пр. ако пишува дека бројот на овие промени (**'#Co-occurring Changes'**) е еднаков на 16, во колоната **"Co-occurring Changes"** се снимени само првите 3. Тоа сега не е проблем но во иднина може да биде доколку сакаме работа на цел геном. Сега за сега тие мутации за варијантите не се потребни бидејќи работиме повеќе на значајни мутации (N501Y, D614G..) на глико-протеинот. 

Сега колоните ни се: 

In [ ]:
combined_csv_data.columns

Index(['Variant', 'Literature Ref', '#Genomes', '#Top Location', '#Top Clade',
       '#Top Lineage', 'Co-occurring Changes', '#Co-occurring Changes',
       'Timestamp', 'Δ#Loc(S)', '#aachanges(C)', '(SxC)'],
      dtype='object')

In [ ]:
combined_csv_data['Co-occurring Changes']

0            Spike_T19R, Spike_E156G, Spike_D950N
1            Spike_S12F, Spike_Q677H, Spike_A899S
2         Spike_A570D, Spike_Y145del, Spike_P681H
3         Spike_A570D, Spike_Y145del, Spike_P681H
4         Spike_A570D, Spike_F490S, Spike_Y145del
                         ...                     
94          Spike_S494P, Spike_I870V, Spike_Y144F
95                       Spike_D614G, NSP12_P323L
96                       Spike_D614G, NSP12_P323L
97          Spike_D614G, Spike_S982A, NSP12_P323L
98    Spike_V143del, Spike_G142del, Spike_L141del
Name: Co-occurring Changes, Length: 10016, dtype: object

Ги отстрануваме сите колони кои моментално не ни требаат: 

In [ ]:
# Зачувување на combined csv data како нов фајл; index=False за да се изгуби "index" колоната
# os.chdir('..')
combined_csv_data = combined_csv_data.drop(["Literature Ref",
                                            "#Genomes",
                                            "#Top Location",
                                            "#Top Clade",
                                            "#Co-occurring Changes",
                                            "Δ#Loc(S)",
                                            "#aachanges(C)",
                                            "(SxC)"], axis=1)

Информации за множеството вака зачувано:

In [ ]:
# dataset info
print("Shape (rows/columns) of new data: ", combined_csv_data.shape)
print("Column names in our new data: ", combined_csv_data.columns.values, "\n")
print("Preview of first 5 rows: ")
print(combined_csv_data.head(5))

combined_csv_data.to_csv('combined_data.csv', index=False)

Shape (rows/columns) of new data:  (10016, 4)
Column names in our new data:  ['Variant' '#Top Lineage' 'Co-occurring Changes' 'Timestamp'] 

Preview of first 5 rows: 
                           Variant  ... Timestamp
0                   452R_478K_681R  ...  May_2021
1            69del_70del_346S_452R  ...  May_2021
2       5F_69del_70del_144del_501Y  ...  May_2021
3     69del_70del_144del_484K_501Y  ...  May_2021
4  5F_69del_70del_144del_490L_501Y  ...  May_2021

[5 rows x 4 columns]


Првата колона кажува кад (позиција на S-протениот) и каква мутација настанала во таа варијанта на битни места (биолошки поврзани со binding места) и на овие ги додаваме сите што табелата ги нуди од **Co-occurring Changes**. Тука веќе имаме множество каде ги знаеме мутациите на новите варијанти и знаеме некаква историја нив т.е. знаеме на која лоза припаѓа. Зошто е ова важно? 


### 1.2: Вчитување на лози


Бидејќи овие **лози** (linages) се битни да се вклучат во текстуалната датотека за учење бидејќи се биолошки "точни". Тука приметив дека нема теорија да се сите 3265 варијанти од различна лоза (како што можеме да видиме од излезот горе варијанта број 0(**69del_70del_439**K), 3 (**477N**) и 4 (**69X_70X_439K**) сите припаѓаат на иста лоза) и поради тоа одлучив да направам како unique листа каде ги имам точен број без повторување (бидејќи не смее за моделот да се повтроруваат): 

In [ ]:
# Читај combined_csv_data from 0-phase
df = pd.read_csv("combined_data.csv")

# Листај ги сите лози
all_lineages = []
for index, row in df.iterrows():
    all_lineages.append(row['#Top Lineage'].split(" ")[1])

# Земи ги сите unique вредности
print("Total number of lineages: ", len(all_lineages), "\n")
all_lineages = list(set(all_lineages))
print("Total number of (unique) lineages: ", len(all_lineages), "\n")

# print(os.getcwd())

lineage_text_file = open("../outbreakinfo2/LINEAGES.txt", "w")
for element in all_lineages:
    lineage_text_file.write(element + "\n")
lineage_text_file.close()

Total number of lineages:  10016 

Total number of (unique) lineages:  276 



> **СПОРЕДБА:** Воочовуваме (можеби подобро кога ќе се прави презентација ќе се направи графикон) дека бројот на лози нови со додавање на САМО еден месец прераснува од: <br>
&#8594; *3 625* на *10 016*  
Додека бројот на уникатни лози од: <br>
&#8594; *178* на *276*


Ова е само индикатор за брзината на мутација на овие +ssRNA и интересни анализи можат да покаже дека преку време (по секој месец) бројот се зголемува повеќе. Ова додавање на последниот месец покажува скоро тродупло зголемување на бројот на варијанти што делува и застрашувачки. 

### 1.3: Анализа на лози

Овој чекор е (идентичен како Фаза 3) е додавање на сите уникатни лози во текст фајл преку <code>.json</code> фајлови. 

Сите мутации на една лоза (реферетни во однос на главниот геном) се запишани како <code>.json</code> фајл и пример може да го земеме најбитниот за да покажеме како изгледа излезот (најбитната лоза = variant of concern) тоа е **B.1.1.7** (Британската варијанта), регистрирани случаеви за оваа се 785 354 што е застрашувачки голем број. [[10]](https://www.cdc.gov/coronavirus/2019-ncov/science/science-briefs/scientific-brief-emerging-variants.html)

In [ ]:
import json

f = open('../outbreakinfo2/B.1.1.7.json', )

# json.load: враќа JSON објект како речник
data = json.load(f)

print(data)

[{'mutation': 's:d614g', 'mutation_count': 781032, 'lineage_count': 785354, 'gene': 'S', 'ref_aa': 'D', 'alt_aa': 'G', 'codon_num': 614, 'codon_end': 'None', 'type': 'substitution', 'prevalence': 0.99449674923665, 'change_length_nt': 'None', 'pangolin_lineage': 'B.1.1.7', 'id': 's_d614g', 'source': None}, {'mutation': 'orf1a:t1001i', 'mutation_count': 779525, 'lineage_count': 785354, 'gene': 'ORF1a', 'ref_aa': 'T', 'alt_aa': 'I', 'codon_num': 1001, 'codon_end': 'None', 'type': 'substitution', 'prevalence': 0.992577869342997, 'change_length_nt': 'None', 'pangolin_lineage': 'B.1.1.7', 'id': 'orf1a_t1001i', 'source': None}, {'mutation': 's:d1118h', 'mutation_count': 779436, 'lineage_count': 785354, 'gene': 'S', 'ref_aa': 'D', 'alt_aa': 'H', 'codon_num': 1118, 'codon_end': 'None', 'type': 'substitution', 'prevalence': 0.9924645446512019, 'change_length_nt': 'None', 'pangolin_lineage': 'B.1.1.7', 'id': 's_d1118h', 'source': None}, {'mutation': 's:a570d', 'mutation_count': 778189, 'lineage_c

Иако вака фајлот изгледа како џибириш програмерски, внатре во него ги има убаво запишано сите мутации бидејќи варијанти не се делат во лози само според мутации во S-протеинот туку и според другите (некои лози / варијанти имаат исти мутации во S-протеинот!). 

Пример сега може да ги читаме мутациите на Британската варијанта: 

In [ ]:
print("B.1.1.7")
i = 0
for mutation in data:
    if mutation['gene'] == 'S':
        i += 1
        
        if mutation['type'] == 'substitution':
          print(str(i) + " mutation is of type - " + mutation['type'], " :", 
                mutation['ref_aa'], 
                mutation['codon_num'],
                mutation['alt_aa'])
          
        else: 
          print(str(i) + " mutation is of type - " + mutation['type'], " :", 
                mutation['ref_aa'])
        
print("\nTotal number of mutations in S-gene: " + str(i), "\n")

B.1.1.7
1 mutation is of type - substitution  : D 614 G
2 mutation is of type - substitution  : D 1118 H
3 mutation is of type - substitution  : A 570 D
4 mutation is of type - substitution  : S 982 A
5 mutation is of type - substitution  : T 716 I
6 mutation is of type - substitution  : P 681 H
7 mutation is of type - substitution  : N 501 Y
8 mutation is of type - deletion  : S:DEL69/70
9 mutation is of type - deletion  : S:DEL144/145

Total number of mutations in S-gene: 9 



Сега за нашиот прв обид за учење со трансформери ние ќе се фокусираме главно на промените што настануваат како резултат на **субституција** пример ќе тргнеме (маскираме) некоја амино киселина (пример доста честа е *D614G* и ако го маскираме пример 614 кодон и моделот ни врати со најголема веројатност дека таму е *G* тогаш научил нешто за нашата проблематика, за ова подолу).

In [ ]:
# Функција за листање на сите мутации во листа од лози
def lineage_mutations(lineage_list):
    for lineage in lineage_list:
        # Бројач за мутации
        i = 0

        # Читање на .json 
        f = open('../outbreakinfo2/' + lineage + '.json', )

        # json.load: враќа JSON објект како речник
        data = json.load(f)

        # Current lineage
        print(lineage)

        # Итерација низ json листа
        for mutation in data:
            if mutation['gene'] == 'S':
                i += 1

                # if mutation['type'] == 'substitution':
                #     print(str(i) + " mutation is of type - " + mutation['type'], " :", 
                #           mutation['ref_aa'], 
                #           mutation['codon_num'],
                #           mutation['alt_aa'])
              
                # else: 
                #     print(str(i) + " mutation is of type - " + mutation['type'], " :", 
                #         mutation['ref_aa'])

          
        print("Total number of mutations in S-gene: " + str(i), "\n")

In [ ]:
from Bio import SeqIO

# Зачувај ги сите lineages како text датотека
lineage_text_file = open("../outbreakinfo2/LINEAGES.txt", "w")
for element in all_lineages:
    lineage_text_file.write(element + "\n")
lineage_text_file.close()

# Лоцирај ги сите Lineages на outbreak info, лоцирај ги нивните мутации
import json

lineage_text_file = open("../outbreakinfo2/LINEAGES.txt", "r")
my_lineages = lineage_text_file.read().splitlines()


# print("-_-_-_-_-_-_-_-_-_-_-_-_-_ LINEAGE ANALYSIS -_-_-_-_-_-_-_-_-_-_-_-_-_ ", "\n")

lineage_mutations(my_lineages[:])

Од анализата горе можеме преку <code>.json</code> фајлови да ги читаме мутациите нивните локации и слично. Сите фајлови се валидни варијанти (некои се варијанти од интерест (intrest), други се варијанти од грижа (concern) но немав време да ги земам сите во предвид за оваа ваза.

### 3.2: Креирање на пептидни ланци за лозите
Сега ги лоциравме сите мутации, знаеме која каде е во која лоза и варијанта сега потребно е да го изградиме нашиот текстуален фајл за подоцна да може моделот да учи. 

Започнуваме со функција која ги запишува сите овие пептидни ланци еднаш за да ги имаме за понатамошни истражување бидејќи ваков датасет можеби постои но не беше јавен (пр. постојат мутациите но ги нема како пептидна секвенца). 

In [ ]:
def create_linage_peptides(S_protein, lineages):
    """

    :param S_protein:
    :param lineages:
    :return:
    """
    linage_aa = []
    for lineage in lineages:
        # Create copy for current linage
        S_copy = S_protein[:]

        # Opening JSON file
        f = open('../outbreakinfo2/' + lineage + '.json', )

        # json.load: returns JSON object as a dictionary
        data = json.load(f)

        # Mutations in linage
        print(lineage)
        for mutation in data:
            if mutation['gene'] == 'S':

                if mutation['type'] == 'substitution':
                    ref_aa = mutation['ref_aa']
                    codon_num = mutation['codon_num'] - 1
                    alt_aa = mutation['alt_aa']

                    print("Mutation is of type - " + mutation['type'], " :",
                          mutation['ref_aa'],
                          mutation['codon_num'],
                          mutation['alt_aa'])

                    # Check if aa on codon is equal to aa on referent genome
                    if S_copy[codon_num] == ref_aa:
                        # Mutate to alternative amino acid
                        S_copy[codon_num] = alt_aa
                        print("Mutated " + ref_aa + " into " + alt_aa)


        S_aa = "".join(S_copy)
        print(S_aa, "\n")
        linage_aa.append(S_aa)

    return linage_aa

In [ ]:
from Bio import SeqIO

os.chdir("/content/drive/MyDrive/data/covid-data/")

for seq_record in SeqIO.parse("sars-cov-2-glycoprotein.fasta", "fasta"):
    print("ID: ", seq_record.id)
    print("Description: Surface glycoprotein [Sars-Cov-2]")
    print("Protein sequence: ", repr(seq_record.seq))
    print("Amino acid length: ", len(seq_record), "\n")


S_region = list(seq_record.seq)

os.chdir("/content/drive/My Drive/data-new/")

lineage_text_file = open("../outbreakinfo2/LINEAGES.txt", "r")
my_lineages = lineage_text_file.read().splitlines()

# Create linage peptides
lineage_peptides = create_linage_peptides(S_region, my_lineages[:])

ID:  YP_009724390.1
Description: Surface glycoprotein [Sars-Cov-2]
Protein sequence:  Seq('MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDL...HYT')
Amino acid length:  1273 

B.1.36.31
Mutation is of type - substitution  : D 614 G
Mutated D into G
Mutation is of type - substitution  : R 102 S
Mutated R into S
Mutation is of type - substitution  : A 222 V
Mutated A into V
MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIISGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSVLEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQGVNCTEV

Сега откако ги добивме сите пептдни ланци интересното е следниот податок: 

In [ ]:
print("Total # of peptides:", len(lineage_peptides))

# Now take unique values
unique_lineages = list(set(lineage_peptides))
print("Total # of peptides after processing all duplicates:", len(unique_lineages))

# Write all peptides into text file
with open('train-1.txt', 'w') as f:
    for item in unique_lineages:
        f.write("%s\n" % item)

Total # of peptides: 276
Total # of peptides after processing all duplicates: 101


По **вадење на сите дупликати** бројот на различни комплетно лози/варијанти се намалува од **276** на **101** бидејќи поголеиот дел се слични. 
Сега веќе имаме **101 влезови** за нашиот трансформер потребно е сега да ги додаеме сите останати варијанти од почетното множество (+да се проверат дупликати). 

Исто во прилог се анализа на фрекфенции на некој од најчестите мутации со супституција што настануваат поради кои и повеќето секвенци се филтрираат со функцијата за дупликати: 

In [ ]:
#@title (Код за фреквенции на ново множество лози)

def percentage(part, whole):
    return 100 * float(part) / float(whole)

lineages = open("train-1.txt", "r")
my_lineages = lineages.read().splitlines()

count_N = 0
count_Y = 0

for lineage in my_lineages:
    if lineage[500] == "N":
        count_N += 1
    elif lineage[500] == 'Y':
        count_Y += 1

print()
print("Count for N on aa position 501:  " + str(count_N),
      f'[{percentage(count_N, count_N + count_Y):.2f}%]')

print("Count for Y on aa position 614: " + str(count_Y),
      f'[{percentage(count_Y, count_N + count_Y):.2f}%]')

count_D = 0
count_G = 0

for lineage in my_lineages:
    if lineage[613] == "D":
        count_D += 1
    elif lineage[613] == 'G':
        count_G += 1

print()
print("Count for D on aa position 614: " + str(count_D),
      f'[{percentage(count_D, count_D + count_G):.2f}%]')

print("Count for G on aa position 614:  " + str(count_G),
      f'[{percentage(count_G, count_D + count_G):.2f}%]')

count_Q = 0
count_H = 0

for lineage in my_lineages:
    if lineage[676] == "Q":
        count_Q += 1
    elif lineage[676] == 'H':
        count_H += 1

print()
print("Count for  Q on aa position 677 is: " + str(count_Q),
      f'[{percentage(count_Q, count_Q + count_H):.2f}%]')

print("Count for H on aa position 677: " + str(count_H),
      f'[{percentage(count_H, count_Q + count_H):.2f}%]')

count_M = 0

for lineage in my_lineages:
    if lineage[0] == "M":
        count_M += 1

print()
print("Count for M on aa position 0: " + str(count_M),
      f'[{percentage(count_M, len(my_lineages)):.2f}%]')



Count for N on aa position 501:  86 [86.87%]
Count for Y on aa position 614: 13 [13.13%]

Count for D on aa position 614: 6 [6.00%]
Count for G on aa position 614:  94 [94.00%]

Count for  Q on aa position 677 is: 96 [95.05%]
Count for H on aa position 677: 5 [4.95%]

Count for M on aa position 0: 101 [100.00%]


Интересен податок е дека овие проценти прилично се сменија без последниот месец. Наликуваа нешто вака: 


In [ ]:
#@title (Код за фреквенции на старо множество лози)

def percentage(part, whole):
    return 100 * float(part) / float(whole)

lineages = open("/content/drive/MyDrive/data/train-1.txt", "r")
my_lineages2 = lineages.read().splitlines()

count_N = 0
count_Y = 0

for lineage in my_lineages2:
    if lineage[500] == "N":
        count_N += 1
    elif lineage[500] == 'Y':
        count_Y += 1

print()
print("Count for N on aa position 501:  " + str(count_N),
      f'[{percentage(count_N, count_N + count_Y):.2f}%]')

print("Count for Y on aa position 614: " + str(count_Y),
      f'[{percentage(count_Y, count_N + count_Y):.2f}%]')

count_D = 0
count_G = 0

for lineage in my_lineages2:
    if lineage[613] == "D":
        count_D += 1
    elif lineage[613] == 'G':
        count_G += 1

print()
print("Count for D on aa position 614: " + str(count_D),
      f'[{percentage(count_D, count_D + count_G):.2f}%]')

print("Count for G on aa position 614:  " + str(count_G),
      f'[{percentage(count_G, count_D + count_G):.2f}%]')

count_Q = 0
count_H = 0

for lineage in my_lineages2:
    if lineage[676] == "Q":
        count_Q += 1
    elif lineage[676] == 'H':
        count_H += 1

print()
print("Count for  Q on aa position 677 is: " + str(count_Q),
      f'[{percentage(count_Q, count_Q + count_H):.2f}%]')

print("Count for H on aa position 677: " + str(count_H),
      f'[{percentage(count_H, count_Q + count_H):.2f}%]')

count_M = 0

for lineage in my_lineages2:
    if lineage[0] == "M":
        count_M += 1

print()
print("Count for M on aa position 0: " + str(count_M),
      f'[{percentage(count_M, len(my_lineages2)):.2f}%]')



Count for N on aa position 501:  57 [87.69%]
Count for Y on aa position 614: 8 [12.31%]

Count for D on aa position 614: 5 [7.46%]
Count for G on aa position 614:  62 [92.54%]

Count for  Q on aa position 677 is: 64 [95.52%]
Count for H on aa position 677: 3 [4.48%]

Count for M on aa position 0: 67 [100.00%]


- Пред се бројот на уникатни лози е нараснат скоро дупло (67 во 101) 
- Најсмртностната мутација <code><b>N501Y</b></code> сеуште е стабилна (прераснала малце од 12% во 13%) 
- Додека останатите "најчести" мутации се горе доле стабилни мали промени со оглед на зголемениот број на лози

### 3.3: Креирање на пептидни ланци за варијантите
Сега следува процесот на наоѓање (добивање) на пептидницте ланци од податоците од GISAID, поточно да ги добиеме сите **"emerging variants"** (множеството во точка 3.1). 

Сега во множеството гледаме колона **"#Top Linage"** ова се мисли на колона која најмногу се совпаѓа доколку се изврши порамнување. 

Пример ако се земе првата варијанта од Април, таа има име: 
> 18F_417T_484K_501Y 

На оваа варијанта најсличен геном со порамнување е **P.1** (каде 6362 од геномите од варијантата се по порамнување најслични со него). Доколку сега ги испечатиме мутациите на **P.1** лозата ќе видиме: 

In [ ]:
f = open('../outbreakinfo2/P.1.json', )
# json.load: returns JSON object as a dictionary
data = json.load(f)

print("P.1")
i = 0
for mutation in data:
    if mutation['gene'] == 'S':
        i+=1
        if mutation['type'] == 'substitution':
            ref_aa = mutation['ref_aa']
            codon_num = mutation['codon_num'] - 1
            alt_aa = mutation['alt_aa']

            print("Mutation is of type - " + mutation['type'], " :",
                  mutation['ref_aa'],
                  mutation['codon_num'],
                  mutation['alt_aa'])
        else:
          print(str(i) + " mutation is of type - " + mutation['type'], " :",
                mutation['ref_aa'])
          
print("\nTotal number of mutations in S-gene in P.1. lineage is: " + str(i), "\n")

P.1
Mutation is of type - substitution  : V 1176 F
Mutation is of type - substitution  : D 614 G
Mutation is of type - substitution  : H 655 Y
Mutation is of type - substitution  : L 18 F
Mutation is of type - substitution  : P 26 S
Mutation is of type - substitution  : T 20 N
Mutation is of type - substitution  : N 501 Y
Mutation is of type - substitution  : E 484 K
Mutation is of type - substitution  : D 138 Y
Mutation is of type - substitution  : T 1027 I
Mutation is of type - substitution  : K 417 T
Mutation is of type - substitution  : R 190 S

Total number of mutations in S-gene in P.1. lineage is: 12 



Гледаме дека сите 4 мутации на нашата варијанта се пристуни кај оваа лоза: 
- L 18 F
- K 417 T
- E 484 K
- N 501 Y

Но, нејзината пептидна низа не е иста бидејќи бројот на мутации кај нашата е 4 додека кај лозата **P.1** бројот на мутации е 12. 

Сега тука е промената од претходната фаза која е клучна. Додека последниот апт ги читавме повеќето од мутациите со своите неколку карактеристични мутации ги додаваме и новите од **"Co-occuring changes"** (барем оние кои се достапни). 

Ја имаме истата функција за читање на име на варијанта само го додаваме и бројот на вакви промени и креиреање на листа од амино киселини: 

In [ ]:
import os
import glob
import pandas as pd
import re

def get_mutations(S_protein, variants):
    var_aa = []
    for name in variants:
        print(name)
        mutations = name.split("_")
        S_copy = S_protein[:]
        for m in range(0, len(mutations)):
            mutations_list = re.split('(\d+)', mutations[m])
            if mutations_list[2] == 'del':
                continue
            else:
                codon_num = int(mutations_list[1]) - 1
                print("Mutating ", S_copy[codon_num], "into", mutations_list[2])
                S_copy[codon_num] = mutations_list[2]

        S_aa = "".join(S_copy)
        print(S_aa[:15] + "..."+S_aa[-1], "\n")
        var_aa.append(S_aa)
    return var_aa

def sequence_compare(seq_a, seq_b):
    len1 = len(seq_a)
    len2 = len(seq_b)
    mismatches = []
    mismatches_count = 0
    print("-^-^-^-^-^-^-^-^-^-^-^-^-^-^-^-^-^-^-^-^-^-^-^-^ MISMATCHING [Custom BLAST] -^-^-^-^-^-^-^-^-^-^-^-^-^-^-^-^-^-^-^-^-^-^-^-^")
    for pos in range(0, min(len1, len2)):
        if seq_a[pos] != seq_b[pos]:
            mismatches.append('|')
            mismatches_count += 1
            print(f"Mismatch {mismatches_count}:", seq_b[pos], pos + 1, seq_a[pos])
        else:
            mismatches.append(' ')
    print("\n", f"Total # of mismatches: {mismatches_count}")
    print()
    print("Lineage")
    print(seq_b)
    print("".join(mismatches))
    print(seq_a)
    print("Variant")

Пример за како ги додаваме и читаме новите промени: 

In [ ]:
f = pd.read_csv("combined_data.csv")

# Top lineage за 1st test case
top_lineage = df['#Top Lineage'][0].split(" ")[1]
print("Top lineage for this variant:", top_lineage)

# Relevant aa changes
relevant_aa = df['Variant'][0].split('_')
print("Relevant aa changes / Antibody cites: ", relevant_aa)

# Co-occurring changes
co_occurring_changes = [ele.split("_")[1] for ele in df['Co-occurring Changes'][0].split(', ')]
print("Co-occurring changes: ", co_occurring_changes)
print()

# Пример за порамнување
seq1_temp = get_mutations(S_region, ["_".join(list(set(relevant_aa+co_occurring_changes)))])
seq2_temp = create_linage_peptides(S_region, [top_lineage])

print(f"Len of variant {relevant_aa}: ", len(seq1_temp[0]))

print(f"Len of lineage {top_lineage}:", len(seq2_temp[0]))

print()
import time
time.sleep(2)
sequence_compare(seq1_temp[0], seq2_temp[0])

Top lineage for this variant: B.1.617.2
Relevant aa changes / Antibody cites:  ['452R', '478K', '681R']
Co-occurring changes:  ['T19R', 'E156G', 'D950N']

T19R_D950N_E156G_452R_478K_681R
Mutating  T into R
Mutating  D into N
Mutating  E into G
Mutating  L into R
Mutating  T into K
Mutating  P into R
MFVFLVLLPLVSSQC...T 

B.1.617.2
Mutation is of type - substitution  : D 614 G
Mutated D into G
Mutation is of type - substitution  : P 681 R
Mutated P into R
Mutation is of type - substitution  : T 19 R
Mutated T into R
Mutation is of type - substitution  : L 452 R
Mutated L into R
Mutation is of type - substitution  : T 478 K
Mutated T into K
Mutation is of type - substitution  : D 950 N
Mutated D into N
MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVD

Сега во воведувањето на овие порамнување (иако се пренеинтутивни и работам на многу подобри интерактивни визуелизации за истите за крајната презентација) можеме да видиме дека имаме не-совпаѓања точно 2 (од кој едната е <code><b>D614G</b></code> која постои 

Сега можеме да модифцираме и направиме нова функција и да започнеме со читање на сите варијанти и принтање на соодветни мутации (зачувување во датотека): 

In [ ]:
from Bio import SeqIO

print("\n", "-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-. Editing changes -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.", "\n")

def get_mutations(S_protein, df_variants):
    var_aa = []
    for i, row in df_variants.iterrows():
        print(df_variants['Variant'][i])
          
        relevant_aa = row['Variant'].split('_')
        print("Relevant aa changes / Antibody cites: ", relevant_aa)

        # Co-occurring changes
        co_occurring_changes = [ele.split("_")[1] for ele in df['Co-occurring Changes'][0].split(', ') if
                        ele.split("_")[0] == 'Spike']
        print("Co-occurring changes (only in Spike): ", co_occurring_changes)

        mutations = list(set(relevant_aa+co_occurring_changes))
        print(mutations)
        S_copy = S_protein[:]
        for m in range(0, len(mutations)):
            mutations_list = re.split('(\d+)', mutations[m])
            if mutations_list[2] == 'del':
                continue
            else:
                codon_num = int(mutations_list[1]) - 1
                print("Mutating ", S_copy[codon_num], "into", mutations_list[2])
                S_copy[codon_num] = mutations_list[2]

        S_aa = "".join(S_copy)
        print(S_aa[:15] + "..."+S_aa[-1], "\n")
        var_aa.append(S_aa)
    return var_aa

df = pd.read_csv("combined_data.csv")

for seq_record in SeqIO.parse("/content/drive/MyDrive/data/covid-data/sars-cov-2-glycoprotein.fasta", "fasta"):
    print("ID: ", seq_record.id)
    print("Description: Surface glycoprotein [Sars-Cov-2]")
    print("Protein sequence: ", repr(seq_record.seq))
    print("Amino acid length: ", len(seq_record), "\n")

S_region = list(seq_record.seq)
variants_peptides = get_mutations(S_region, df)
unique_peptides = set(variants_peptides)

print("Total # of peptides:", len(variants_peptides))
print("Total # of unique peptides:", len(unique_peptides))

# Write all peptides into text file
with open('train-2.txt', 'w') as f:
    for item in unique_peptides:
        f.write("%s\n" % item)




Streaming output truncated to the last 5000 lines.
Mutating  M into I
Mutating  E into K
MFVFLVLLPLVSSQC...T 

243X_477N
Relevant aa changes / Antibody cites:  ['243X', '477N']
Co-occurring changes (only in Spike):  ['T19R', 'E156G', 'D950N']
['477N', 'T19R', 'D950N', 'E156G', '243X']
Mutating  S into N
Mutating  T into R
Mutating  D into N
Mutating  E into G
Mutating  A into X
MFVFLVLLPLVSSQC...T 

5F_440K
Relevant aa changes / Antibody cites:  ['5F', '440K']
Co-occurring changes (only in Spike):  ['T19R', 'E156G', 'D950N']
['5F', 'T19R', 'D950N', 'E156G', '440K']
Mutating  L into F
Mutating  T into R
Mutating  D into N
Mutating  E into G
Mutating  N into K
MFVFFVLLPLVSSQC...T 

493K_501T
Relevant aa changes / Antibody cites:  ['493K', '501T']
Co-occurring changes (only in Spike):  ['T19R', 'E156G', 'D950N']
['T19R', '493K', 'D950N', 'E156G', '501T']
Mutating  T into R
Mutating  Q into K
Mutating  D into N
Mutating  E into G
Mutating  N into T
MFVFLVLLPLVSSQC...T 

69del_70del_439K_44

**СПОРЕДБА**: 
- Без Мај месец: 3 265 варијанти, само 620 уникатни 
- Со мај месец: 10 016 варијанти, само 2152 уникатни

Ова е одлично бидејќи бројот се зголеми и сега малку по малку имаме по-реално сценарио. Овие сега може да ги запиешеме во нова датотека со цел да ги чуваме за понатаму: 

In [ ]:
# Запиши ги сите пептдити на варијантите во една текстуална датотека
with open('train-2.txt', 'w') as f:
    for item in unique_peptides:
        f.write("%s\n" % item)

### 3.4: Креирање на пробен dataset
Сега имаме 2 текстуални датотеки: 
- <code>train-1.txt</code> (вкупно 101): сите пептидни ланци на лози (linages)
- <code>train-2.txt</code> (вкупно 2152): сите пептни ланци на emerging варијанти 

Сега потребно е да ги прочитаме и уште еднаш за секој случај да провериме дали има дупликати и да провериме дали должините ни се исти на инстанци од двете листи: 

In [ ]:
# Читај лози 
import os
# os.chdir("./drive/MyDrive/data")
lineages = open("train-1.txt", "r")
my_lineages = lineages.read().splitlines()

# Читај варијанти
variants = open("train-2.txt", "r")
my_variants = variants.read().splitlines()

# Споредба на должини на инстаци од двете листи
# print("Length of variant: ", len(my_variants[0]))
# print("Length of lineage: ", len(my_lineages[1]))

# print()

# Проверка за дупликати
all_variants = my_variants + my_lineages
print("Total # of variants + lineages:", len(all_variants))
unique_variants = list(set(all_variants))
print("Total # of unique variants + lineages:", len(unique_variants))

# Запиши ги сите варијанти/лози во една датотека
with open('train.txt', 'w') as f:
    for item in unique_variants:
        f.write("%s\n" % item)

Total # of variants + lineages: 2253
Total # of unique variants + lineages: 2253


Финалното множество ни е запишано во <code>train.txt</code> со број на примероци **2253**. 

## 2: Трансформери

In [ ]:
#@title (Python библиотеки)
!pip uninstall -y tensorflow
!pip install transformers
# !pip list | grep -E 'transformers|tokenizers'

### 2.1: Setup на модел

Конфигурација на tokenizer и градење на речник со големина 20 (20 амино киселини): 

In [ ]:
%%time 
from pathlib import Path
import os
from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path("../transformer_data/").glob("**/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=20, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])
vocab_size = tokenizer.get_vocab_size()
print(vocab_size)
print(tokenizer.get_vocab())

261
{'į': 240, '<mask>': 4, '(': 12, 'T': 56, '½': 126, 'í': 174, 'ł': 259, 'Ń': 260, '®': 111, '#': 7, 'Č': 205, 'Ú': 155, '4': 24, ']': 65, 'Ò': 147, '%': 9, 'ä': 165, 'ě': 220, 'Ě': 219, 'ó': 180, 'ŀ': 257, 'ì': 173, '&': 10, 'Ė': 215, 'w': 91, '+': 15, 'Ł': 258, 'ĩ': 234, 'é': 170, 'Ì': 141, 'ď': 208, 'Ć': 199, 'ı': 242, '¦': 104, 'R': 54, '¢': 100, 'Ü': 157, 'ċ': 204, 'ħ': 232, 'ê': 171, 'G': 43, 'ë': 172, 'Õ': 150, 'x': 92, 'ĉ': 202, 'Ð': 145, 'Ħ': 231, 'Â': 131, 'ę': 218, '*': 14, '¶': 119, '£': 101, 'f': 74, 'Ñ': 146, '¿': 128, 'P': 52, 'ô': 181, 'F': 42, 'ý': 190, 'ĸ': 249, 'Ę': 217, 'û': 188, 'ĕ': 214, 'Ą': 197, 'U': 57, '×': 152, '6': 26, '¡': 99, 'Þ': 159, 'ļ': 253, 'Ĝ': 221, '¯': 112, 't': 88, 'r': 86, '|': 96, 'i': 77, 'Ŀ': 256, 'Ä': 133, 'â': 163, 'Ğ': 223, 'ß': 160, '»': 124, '\\': 64, 'ĺ': 251, '¾': 127, 'å': 166, 'ğ': 224, 'á': 162, 'è': 169, 'Ĳ': 243, '©': 107, 'Ï': 144, 'ĵ': 246, '¥': 103, 'K': 47, '§': 105, '~': 98, 'c': 71, 'Ù': 154, '=': 33, 'Ç': 136, 'ç': 168, '

Креиеање на вокабулар и фајл за спојки: 

In [ ]:
# !mkdir EsperBERTo
tokenizer.save_model("EsperBERTo")

['EsperBERTo/vocab.json', 'EsperBERTo/merges.txt']

Резултатот се добива пребрзо бидејќи и множеството не е преголемо. Сега имаме: 
- vocab.json: листа од најчести токени рангирани по фреквенција 
- merges.txt: листа од спојки (merges)

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./EsperBERTo/vocab.json",
    "./EsperBERTo/merges.txt",
)

In [ ]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=2048)

Пример за токенизација на некоја наша рандом секвенца од амино киселини: 

In [ ]:
print(type(my_lineages[0]))

<class 'str'>


In [ ]:
print(my_lineages[12])
x = my_lineages[12]
print(x[-1])
print(tokenizer.encode(x).tokens[-2])
print(tokenizer.encode(x))
tokenizer.encode(x).tokens[-1]

MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSVLEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQGVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDKVEAEVQIDRLITGR

'</s>'

Гледаме од претходниот излез дека добро се извршува самата токенизација (1273 е должината на пептиднот ланец на S-генот + 2 токени за крај и почеток). Исто така последната буква ако е T на токенот да е претпоследна бидејќи последна е самиот токен за крај. 

In [ ]:
# tokenizer.encode(x).tokens

Конфигурација на графичка и останати библиотеки: 

In [ ]:
# Проверка за дали имаме поврзано GPU
!nvidia-smi

Wed Jun 23 14:27:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Проверка дека PyTorch ја гледа графичката
import torch
torch.cuda.is_available()

True

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=vocab_size,
    max_position_embeddings=2048,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [ ]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./EsperBERTo", max_len=2048)

In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [ ]:
model.num_parameters()
# => 44 million parameters

44895237

In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="train.txt",
    block_size=1273
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 2.73 s, sys: 382 ms, total: 3.11 s
Wall time: 2.32 s


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
# Split dataset X% train, (1-X)% test ----- > 85% train, 15% test
X = 85
train_csv = dataset[int((1-float(X)/100) * len(dataset)):]
eval_csv = dataset[:int((1-float(X)/100) * len(dataset))]
print(f"# of training samples:  {len(train_csv)}")
print(f'# of evaluation samples: {len(eval_csv)}')

# of training samples:  1916
# of evaluation samples: 337


In [ ]:
from transformers import Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir="./EsperBERTo",
    evaluation_strategy = "epoch",
    overwrite_output_dir=True,
    num_train_epochs=20,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_csv ,
    eval_dataset=eval_csv,
    # compute_metrics=compute_metrics,
)

### 2.2: Тренирање

In [ ]:
%%time
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.213829
2,0.482400,0.177829
3,0.182700,0.112041
4,0.126900,0.058564
5,0.066900,0.031061
6,0.044000,0.036757
7,0.037900,0.026109
8,0.033700,0.021049
9,0.032000,0.037437
10,0.034200,0.024297


CPU times: user 2h 7min 33s, sys: 17.2 s, total: 2h 7min 50s
Wall time: 2h 8min 17s


TrainOutput(global_step=9580, training_loss=0.06701282110990711, metrics={'train_runtime': 7697.6671, 'train_samples_per_second': 4.978, 'train_steps_per_second': 1.245, 'total_flos': 1.314030431029392e+16, 'train_loss': 0.06701282110990711, 'epoch': 20.0})

In [ ]:

trainer.evaluate()

{'epoch': 20.0,
 'eval_loss': 0.017011556774377823,
 'eval_runtime': 23.5974,
 'eval_samples_per_second': 14.281,
 'eval_steps_per_second': 1.822}

Во повеќето литератури доколку <code><b>Training Loss</b></code> и <code><b>Validation Loss </b></code> се близу тогаш во теорија моделот добро учел. На почетокот малце има bump на 2ра епоха но потоа се исправува. 

In [ ]:
trainer.save_model("./EsperBERTo")

### 2.3: Тестирање (маскирање на мутации)

In [ ]:
#@title (Фреквенциска анализа пред мутирање)
import os
import glob
import pandas as pd

# Читај ги сите фајлови од data-new фолдерот
os.chdir("/content/drive/MyDrive/data-new")

print("\n\n-=-=-=-=-=-=-=-=-=-=-=- Frequency analysis -=-=-=-=-=-=-=-=-=-=-=-")


variants = open("train.txt", "r")
unique_variants = variants.read().splitlines()
print("Total # of variants + lineages:", len(unique_variants))

def percentage(part, whole):
    return 100 * float(part) / float(whole)

# Мутација n501y
count_N = 0
count_Y = 0

for lineage in unique_variants:
    if lineage[500] == "N":
        count_N += 1
    elif lineage[500] == 'Y':
        count_Y += 1

print()
print(" -=-=-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-=-=-=-=-=- Mutation N501Y ")
print("Count N for codon 501 is: " + str(count_N),
      f'[{percentage(count_N, count_N + count_Y):.2f}%]')

print("Count Y for codon 501 is: " + str(count_Y),
      f'[{percentage(count_Y, count_N + count_Y):.2f}%]')

# Мутација d614g
count_D = 0
count_G = 0

for lineage in unique_variants:
    if lineage[613] == "D":
        count_D += 1
    elif lineage[613] == 'G':
        count_G += 1

print()
print(" -=-=-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-=-=-=-=-=- Mutation D614G ")
print("Count D for codon 614 is: " + str(count_D),
      f'[{percentage(count_D, count_D + count_G):.2f}%]')

print("Count G for codon 614 is: " + str(count_G),
      f'[{percentage(count_G, count_D + count_G):.2f}%]')

# Мутација q677h
count_Q = 0
count_H = 0

for lineage in unique_variants:
    if lineage[676] == "Q":
        count_Q += 1
    elif lineage[676] == 'H':
        count_H += 1

print()
print(" -=-=-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-=-=-=-=-=-Mutation Q677H ")
print("Count Q for codon 677 is: " + str(count_Q),
      f'[{percentage(count_Q, count_Q + count_H):.2f}%]')

print("Count H for codon 677 is: " + str(count_H),
      f'[{percentage(count_H, count_Q + count_H):.2f}%]')




-=-=-=-=-=-=-=-=-=-=-=- Frequency analysis -=-=-=-=-=-=-=-=-=-=-=-
Total # of variants + lineages: 2253

 -=-=-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-=-=-=-=-=- Mutation N501Y 
Count N for codon 501 is: 1406 [63.91%]
Count Y for codon 501 is: 794 [36.09%]

 -=-=-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-=-=-=-=-=- Mutation D614G 
Count D for codon 614 is: 2158 [95.83%]
Count G for codon 614 is: 94 [4.17%]

 -=-=-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-=-=-=-=-=-Mutation Q677H 
Count Q for codon 677 is: 2248 [99.78%]
Count H for codon 677 is: 5 [0.22%]


In [ ]:
from transformers import pipeline
import os

fill_mask = pipeline(
    "fill-mask",
    model="./EsperBERTo",
    tokenizer="./EsperBERTo"
)

# Mask: 614 - очекуваме 95.83% D; 4.17% G
# =>
test = list(unique_variants[0])
test[613] = '<mask>'
B_TEMP_masked = "".join(test)
fill_mask(B_TEMP_masked)

[{'score': 0.9609606266021729,
  'sequence': 'MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVXYHKNNKSWMESGFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATKFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVKGFNCYFPLQSYGFQPTYGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQNVVNQ

In [ ]:
# Mask: 0 - очекуваме 100% M
# =>
test = list(unique_variants[-20])
test[0] = '<mask>'
B_TEMP_masked  = "".join(test)
fill_mask(B_TEMP_masked )

[{'score': 0.9998341798782349,
  'sequence': 'MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESGFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYQYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQNVVNQ

In [ ]:
# Mask: 501 - очекуваме 63% N, 36% Y
# =>
test1 = list(unique_variants[-39])

test1[500] = '<mask>'
B_TEMP_masked  = "".join(test1)
fill_mask(B_TEMP_masked)

[{'score': 0.6699199676513672,
  'sequence': 'MFVFLVLLPLVSSQCVNFRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESGFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGNIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVKGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKHFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQNVVNQ

In [ ]:
# Mask: 676 - очекуваме 99.78% Q, 0.22% H
# =>
test = list(unique_variants[115])
test[676] = '<mask>'
B_TEMP_masked  = "".join(test)
fill_mask(B_TEMP_masked )

[{'score': 0.998786985874176,
  'sequence': 'MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESGFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLSGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQYSLSSTASALGKLQNVVNQN